# Import dependencies

In [17]:
import pandas as pd
import numpy as np
import ast
import geopandas as gpd
import ee
# ee.Authenticate()
# ee.Initialize()
import os
import time
from shapely.geometry import LineString

In [6]:
gdf_combined_split_clean = gpd.read_feather('data/roads_sub_split_clean.feather')

# Extract pixel values and save to connected Google drive

In [13]:
# Function to extract pixel values from an image
def extract_values(feature, image, sensor_name, year, band, sl):
    image = image.select(band)
    sampled = image.sample(region=feature.geometry(), scale=sl)
    sampled_values = sampled.aggregate_array(band)
    return ee.Feature(None, {
        'SampledValues': sampled_values,
        'Sensor': sensor_name,
        'Status': feature.get('Status'), 
        'Unique_ID': feature.get('Unique_ID'),
        'Unique_ID_Fraction': feature.get('Unique_ID_Fraction'),
        'Year': year,
        'Band': band
    })

# Function to scale DN to reflectance for Landsat 8
def scale_to_reflectance_l8(img):
    optical_bands = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    return img.addBands(optical_bands, None, True)

# Function to scale DN to reflectance for Sentinel-2
def scale_to_reflectance_s2(img):
    return img.multiply(0.0001)

In [16]:
# takes a while to download for regional studies. Takes about 3 hours for Rwanda from 2013 to 2023

In [15]:
new_gdf = gdf_combined_split_clean # gdf_combined_split_clean_rw

# Load image collections with cloud filters applied
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filter(ee.Filter.lt('CLOUD_COVER', 20))
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Define start and end years
start_year_l8 = 2023 # changed 2013 to 2023 when creating multitemporal road quality maps for Rwanda
end_year = 2023
start_year_s2 = 2018

batch_size = 500  # Adjust this value if export fails because of memory error


# Iterate over each year to process data
for year in range(start_year_l8, end_year + 1):
    print(f"Processing year: {year}")
    start_date, end_date = f"{year}-01-01", f"{year}-12-31"
    
    # Prepare Landsat 8 yearly composite
    l8_yearly = l8.filterDate(start_date, end_date).map(scale_to_reflectance_l8).median()
    l8_yearly = l8_yearly.addBands(l8_yearly.expression(
        "1 - ((3 * min(min(SR_B6, SR_B5), SR_B2)) / (SR_B6 + SR_B5 + SR_B2))",
        {'SR_B6': l8_yearly.select('SR_B6'), 'SR_B5': l8_yearly.select('SR_B5'), 'SR_B2': l8_yearly.select('SR_B2')}
    ).rename('Road_Index')).addBands(l8_yearly.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI'))

    # Process batches of features
    for start in range(0, len(new_gdf), batch_size):
        end = start + batch_size
        batch = new_gdf.iloc[start:end]
        
        # Create a feature collection for the batch
        features = [ee.Feature(ee.Geometry.MultiLineString(row.geometry.__geo_interface__['coordinates']), 
                               {'Status': row['Status'], 'Unique_ID': row['Unique_ID'], 'Unique_ID_Fraction': row['Unique_ID_Fraction']}) 
                    for _, row in batch.iterrows()]
        fc = ee.FeatureCollection(features)
        
        # Process each band for Landsat 8
        tasks = []
        for band in ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']:
            l8_extracted = fc.map(lambda feature: extract_values(feature, l8_yearly, 'Landsat', year, band, 30))
            description = f'{band}_batch_{start}_year_{year}'
            task = ee.batch.Export.table.toDrive(
                collection=l8_extracted,
                description=description,
                folder='landsat_test',
                fileNamePrefix=description,
                fileFormat='CSV'
            )
            task.start()
            tasks.append(task)
        
        # Monitor task status
        for task in tasks:
            while task.status()['state'] in ['READY', 'RUNNING']:
                time.sleep(10)
        print("All Landsat tasks completed for the current batch.")
        
        # Prepare and process Sentinel-2 data if applicable
        if year >= start_year_s2:
            s2_yearly = s2.filterDate(start_date, end_date).map(scale_to_reflectance_s2).median()
            s2_yearly = s2_yearly.addBands(s2_yearly.expression(
                "1 - ((3 * min(min(B11, B8), B2)) / (B11 + B8 + B2))",
                {'B11': s2_yearly.select('B11'), 'B8': s2_yearly.select('B8'), 'B2': s2_yearly.select('B2')}
            ).rename('Road_Index')).addBands(s2_yearly.normalizedDifference(['B11', 'B8']).rename('NDBI'))

            # Process each band for Sentinel-2
            for band in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']:
                s2_extracted = fc.map(lambda feature: extract_values(feature, s2_yearly, 'Sentinel', year, band, 10))
                description = f'{band}_batch_{start}_year_{year}'
                task = ee.batch.Export.table.toDrive(
                    collection=s2_extracted,
                    description=description,
                    folder='sentinel_test',
                    fileNamePrefix=description,
                    fileFormat='CSV'
                )
                task.start()

Processing year: 2023
All Landsat tasks completed for the current batch.


In [ ]:
# download csvs and save in a folder 

# Combine CSVs 

In [ ]:
# Set the year and obtain unique countries
year = '2019' # or 2023
unique_countries = gdf_combined_split_clean['NAME_EN'].unique()

# Folders
folders = {
    'Landsat': 'data' + year + '/landsat',
    'Sentinel': 'data' + year + '/sentinel'
}

# Loop through both folders
for sensor, folder in folders.items():
    # List all CSV files in the folder
    csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]

    # Initialize an empty list to store individual DataFrames
    dataframes = []

    # Loop through each CSV file
    for file in csv_files:
        # Construct the full file path
        file_path = os.path.join(folder, file)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Drop unnecessary columns
        df = df.drop(columns=['.geo', 'Year', 'system:index'])
        dataframes.append(df)
    
    # Concatenate all DataFrames into a single DataFrame for the sensor
    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df = combined_df.drop_duplicates()

    # Process each country in the combined DataFrame
    for country in unique_countries:
        # Get unique IDs for the country
        rw_ids = gdf_combined_split_clean[gdf_combined_split_clean.NAME_EN == country].Unique_ID.unique()
        
        # Filter the DataFrame for the current country's unique IDs
        df_filtered = combined_df[combined_df.Unique_ID.isin(rw_ids)]
        
        # Convert 'SampledValues' column from string to list and explode it
        df_filtered['SampledValues'] = df_filtered['SampledValues'].apply(ast.literal_eval)
        df_filtered = df_filtered.explode('SampledValues')
        
        # Create a new unique ID column and a counter for pivoting
        df_filtered['Unique_ID_new'] = df_filtered['Unique_ID'].astype(str) + '_' + df_filtered['Unique_ID_Fraction'].astype(str)
        df_filtered['Counter'] = df_filtered.groupby(['Unique_ID_new', 'Band']).cumcount()

        # Pivot the DataFrame and merge with other relevant columns
        pivot_df = df_filtered.pivot(index=['Unique_ID_new', 'Counter'], columns='Band', values='SampledValues').reset_index()
        other_columns_df = df_filtered[['Unique_ID_new', 'Sensor', 'Status']].drop_duplicates()
        pivot_df = pivot_df.merge(other_columns_df, on='Unique_ID_new', how='left').drop(columns='Counter')

        # Save the pivoted DataFrame as a Feather file
        output_filename = f'data/pivoted/{country}_{sensor}_{year}.feather'
        pivot_df.to_feather(output_filename)

In [19]:
#test